<a href="https://colab.research.google.com/github/summerofstart/CORE-shader/blob/main/SUMMERbinary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio trimesh

In [ ]:
import gradio as gr
import numpy as np
import trimesh
import json
from datetime import datetime
from PIL import Image
import base64
import io
import os
import tempfile
from sklearn.cluster import KMeans

def load_obj(file):
    return trimesh.load_mesh(file.name)

def voxelize_mesh(mesh, resolution):
    voxels = mesh.voxelized(pitch=0.8 / resolution)
    return voxels

def limit_blocks(voxels, max_blocks=30000):
    total_blocks = np.sum(voxels.matrix)
    if total_blocks <= max_blocks:
        return voxels

    flat_indices = np.random.choice(np.flatnonzero(voxels.matrix), size=max_blocks, replace=False)
    new_matrix = np.zeros_like(voxels.matrix)
    new_matrix.flat[flat_indices] = 1

    return trimesh.voxel.VoxelGrid(new_matrix)

def convert_to_serializable(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

def optimize_texture(texture_file, target_size):
    if texture_file is None:
        default_texture = Image.new('RGBA', (target_size, target_size), (128, 128, 128, 255))
        buffered = io.BytesIO()
        default_texture.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        return default_texture, img_base64

    with Image.open(texture_file.name) as img:
        img = img.convert('RGBA')
        img = img.resize((target_size, target_size), Image.LANCZOS)

        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        return img, img_base64

def cluster_voxels(voxels, n_clusters=10):
    """Cluster voxels into groups for easier animation."""
    coords = np.argwhere(voxels.matrix)
    if len(coords) < n_clusters:
        n_clusters = len(coords)
    kmeans = KMeans(n_clusters=n_clusters, n_init=10)
    kmeans.fit(coords)
    return kmeans.labels_, kmeans.cluster_centers_, coords

def create_bone_structure(cluster_labels, cluster_centers):
    """Create a hierarchical bone structure based on clustered voxels."""
    bones = []
    for i, center in enumerate(cluster_centers):
        bone = {
            "name": f"bone_{i}",
            "pivot": center.tolist(),
            "cubes": []
        }
        bones.append(bone)
    return bones

def assign_voxels_to_bones(coords, cluster_labels, bones):
    """Assign voxels to their corresponding bones."""
    for i, (x, y, z) in enumerate(coords):
        bone_index = cluster_labels[i]
        cube = {
            "origin": [int(x), int(y), int(z)],
            "size": [1, 1, 1],
            "uv": {
                "north": {"uv": [0, 0], "uv_size": [1, 1]},
                "east": {"uv": [0, 0], "uv_size": [1, 1]},
                "south": {"uv": [0, 0], "uv_size": [1, 1]},
                "west": {"uv": [0, 0], "uv_size": [1, 1]},
                "up": {"uv": [0, 0], "uv_size": [1, 1]},
                "down": {"uv": [0, 0], "uv_size": [1, 1]}
            }
        }
        bones[bone_index]["cubes"].append(cube)
    return bones

def create_blockbench_json(voxels, block_count, scale, texture_data, model_format):
    cluster_labels, cluster_centers, coords = cluster_voxels(voxels)
    bones = create_bone_structure(cluster_labels, cluster_centers)
    bones = assign_voxels_to_bones(coords, cluster_labels, bones)

    if model_format == "java":
        elements = []
        for bone in bones:
            for cube in bone["cubes"]:
                element = {
                    "name": f"{bone['name']}_{len(elements)}",
                    "from": [cube["origin"][0], cube["origin"][1], cube["origin"][2]],
                    "to": [cube["origin"][0] + 1, cube["origin"][1] + 1, cube["origin"][2] + 1],
                    "faces": {
                        "north": {"uv": [0, 0, 1, 1], "texture": "#texture"},
                        "east": {"uv": [0, 0, 1, 1], "texture": "#texture"},
                        "south": {"uv": [0, 0, 1, 1], "texture": "#texture"},
                        "west": {"uv": [0, 0, 1, 1], "texture": "#texture"},
                        "up": {"uv": [0, 0, 1, 1], "texture": "#texture"},
                        "down": {"uv": [0, 0, 1, 1], "texture": "#texture"}
                    }
                }
                elements.append(element)

        blockbench_data = {
            "meta": {
                "format_version": "4.5",
                "model_format": "generic",
                "box_uv": False
            },
            "name": "Converted OBJ Model",
            "parent": "block/block",
            "ambientocclusion": True,
            "front_gui_light": False,
            "visible_box": [1, 1, 0],
            "variable_placeholders": "",
            "variable_placeholder_buttons": [],
            "resolution": {
                "width": 16,
                "height": 16
            },
            "elements": elements,
            "outliner": [{"name": "root", "origin": [0, 0, 0], "color": 0, "uuid": ""}],
            "textures": [
                {
                    "name": "texture",
                    "folder": "block",
                    "namespace": "custom",
                    "id": "texture",
                    "particle": True,
                    "render_mode": "default",
                    "visible": True,
                    "mode": "bitmap",
                    "saved": True,
                    "uuid": "2185a249-2912-550f-eaf4-b511f635b1a5",
                    "source": f"data:image/png;base64,{texture_data}"
                }
            ]
        }
    elif model_format == "bedrock":
        # Bedrock版の処理は変更なし
        blockbench_data = {
            "format_version": "1.12.0",
            "minecraft:geometry": [
                {
                    "description": {
                        "identifier": "geometry.converted_obj_model",
                        "texture_width": 16,
                        "texture_height": 16,
                        "visible_bounds_width": 2,
                        "visible_bounds_height": 3.5,
                        "visible_bounds_offset": [0, 1.25, 0]
                    },
                    "bones": bones
                }
            ]
        }

    return blockbench_data

def convert_obj_to_blockbench_json(obj_file, texture_file, resolution, texture_size, scale, model_format):
    try:
        mesh = load_obj(obj_file)
        voxels = voxelize_mesh(mesh, resolution)
        limited_voxels = limit_blocks(voxels, max_blocks=30000)

        optimized_texture, texture_base64 = optimize_texture(texture_file, texture_size)

        blockbench_data = create_blockbench_json(limited_voxels, np.sum(limited_voxels.matrix), scale, texture_base64, model_format)

        json_string = json.dumps(blockbench_data, indent=2)

        with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as temp_file:
            temp_file.write(json_string)
            json_file_path = temp_file.name

        with tempfile.NamedTemporaryFile(mode='wb', suffix='.png', delete=False) as temp_file:
            optimized_texture.save(temp_file, format="PNG")
            texture_file_path = temp_file.name

        return [json_file_path, texture_file_path, f"Block count: {np.sum(limited_voxels.matrix)}"]

    except Exception as e:
        return [None, None, f"Error occurred: {str(e)}"]

iface = gr.Interface(
    fn=convert_obj_to_blockbench_json,
    inputs=[
        gr.File(label="Upload OBJ File"),
        gr.File(label="Upload Texture File (optional)"),
        gr.Slider(minimum=8, maximum=100, step=1, label="Voxel Resolution", value=16),
        gr.Slider(minimum=16, maximum=512, step=16, label="Texture Size", value=256),
        gr.Slider(minimum=0.3, maximum=2, step=0.1, label="Block Scale", value=1),
        gr.Radio(["java", "bedrock"], label="Model Format", value="java")
    ],
    outputs=[
        gr.File(label="Download Blockbench JSON File"),
        gr.File(label="Download Optimized Texture"),
        gr.Textbox(label="Block Count")
    ],
    title="Advanced OBJ to Blockbench JSON Converter with Animation Support",
    description="Upload an OBJ file and optionally a texture file to convert it to a Blockbench JSON format with optimized texture and improved structure for animations. The model will be limited to 30000 blocks and automatically clustered for easier animation. Choose between Java Edition and Bedrock Edition formats.",
    allow_flagging="never"
)

if __name__ == "__main__":
    iface.launch(debug=True)